In [1]:
import os
from multiprocessing import cpu_count
import numpy as np
import shutil
import paddle
import paddle.fluid as fluid

In [2]:
def create_data_list(data_root_path):
    #创建test_list 和 train_list
    with open(data_root_path+'test_list.txt','w') as f:
        pass
    with open(data_root_path+'train_list.txt','w') as f:
        pass

    with open(os.path.join(data_root_path,'dict_txt.txt'),'r',encoding='utf-8') as f_data:
        dict_txt = eval(f_data.readlines()[0])

    with open(os.path.join(data_root_path,'news_classify_data.txt'),'r',encoding='utf-8') as f_data:
        lines = f_data.readlines()
    i = 0
    for lines in lines:
        title = lines.split('_!_')[-1].replace('\n','')
        l = lines.split('_!_')[1]
        labs = ""
        if i %10 == 0:
            with open(os.path.join(data_root_path,'test_list.txt'),'a',encoding='utf-8') as f_test:
                for s in title:
                    lab = str(dict_txt[s])
                    labs = labs + lab + ','
                labs = labs[:-1]
                labs = labs + '\t' + l + '\n'
                f_test.write(labs)
        else:
            with open(os.path.join(data_root_path,'train_list.txt'),'a',encoding='utf-8') as f_train:
                for s in title:
                    lab = str(dict_txt[s])
                    labs = labs + lab + ','
                labs = labs[:-1]
                labs = labs + '\t' + l + '\n'
                f_train.write(labs)
        i+=1
    print("数据列表生成完成")

In [3]:
def create_dict(data_path, dict_path):
    dict_set = set()

    with open(data_path,'r',encoding='utf-8') as f:
        lines = f.readlines()
    for line in lines:
        title = line.split('_!_')[-1].replace('\n','')
        for s in title:
            dict_set.add(s)

    dict_list = []
    i = 0
    for s in dict_set:
        dict_list.append([s, i])
        i+=1
    dict_txt = dict(dict_list)
    end_dict = {"<unk>": i}
    dict_txt.update(end_dict)
    with open(dict_path, 'w', encoding='utf-8') as f:
        f.write(str(dict_txt))

    print("数据字典生成完成")

In [4]:
#获取字典长度
def get_dict_len(dic_path):
    with open(dic_path,'r',encoding='utf-8') as f:
        line = eval(f.readlines()[0])
    return len(line.keys())

In [5]:
data_root_path = 'dataset1/'
data_path = os.path.join(data_root_path,'news_classify_data.txt')
dict_path = os.path.join(data_root_path,'dict_txt.txt')

create_dict(data_path,dict_path)
create_data_list(data_root_path)

数据字典生成完成
数据列表生成完成


In [6]:
#训练、测试数据的预处理
def data_mapper(sample):
    data,label = sample
    data = [int(data) for data in data.split(',')]
    return data,int(label)

def train_reader(train_list_path):
    def reader():
        with open(train_list_path,'r') as f:
            lines = f.readlines()
            np.random.shuffle(lines)
            for line in lines:
                data, label = line.split('\t')
                yield data,label
    return paddle.reader.xmap_readers(data_mapper,reader,cpu_count,1024)

#创建数据读取器：
def test_reader(test_list_path):
    def reader():
        with open(test_list_path,'r') as f:
            lines = f.readlines()
            for line in lines:
                data, label = line.split('/t')
                yield data,label
    return paddle.reader.xmap_readers(data_mapper,reader,cpu_count,1024)


In [7]:
#创建卷积神经网络
def CNN_net(data,dict_dim,class_dim=10,emb_dim=128,hid_dim=128,hid_dim2=98):
    #embedding
    emb = fluid.layers.embedding(input=data,size=[dict_dim,emb_dim])

    #
    conv_3 = fluid.nets.sequence_conv_pool(input=emb,num_filters=hid_dim,filter_size=3,act='tanh',pool_type='sqrt')

    conv_4 = fluid.nets.sequence_conv_pool(input=emb,num_filters=hid_dim2,filter_size=4,act='tanh',pool_type='sqrt')

    output = fluid.layers.fc(input=[conv_3,conv_4],size=class_dim,act='softmax')
    return output

In [8]:
#   正确的label
paddle.enable_static()

words = fluid.layers.data(name='words',shape=[1],dtype='int64',lod_level=1)
label = fluid.layers.data(name='label',shape=[1],dtype='int64')

dict_dim = get_dict_len('dataset1/dict_txt.txt')

model = CNN_net(words,dict_dim=dict_dim)
#计算损失
cost = fluid.layers.cross_entropy(input=model,label=label)

avg_cost = fluid.layers.mean(cost)

acc = fluid.layers.accuracy(input=model, label=label)

#预测程序,测试集上run
test_program = fluid.default_main_program().clone(for_test=True)

#优化方法
optimizer = fluid.optimizer.AdagradOptimizer(learning_rate=0.002)
opt = optimizer.minimize(avg_cost)

place = fluid.CUDAPlace(0)

exe = fluid.Executor(place)

exe.run(fluid.default_startup_program())

[]

In [9]:
train_reader = paddle.batch(reader=train_reader('dataset1/train_list.txt'),batch_size=30)
test_reader = paddle.batch(reader=test_reader('dataset1/test_list.txt'),batch_size=30)

In [10]:
feeder = fluid.DataFeeder(place=place,feed_list=[words,label])

In [11]:
epochs = 10
model_save_dir = '/dataset1/infer_model/'
#开始训练
for pass_id in range(epochs):
    for batch_id,data in enumerate(train_reader()):
        train_cost,train_acc = exe.run(program=fluid.default_main_program(),
                                       feed=feeder.feed(data),
                                       fetch_list=[avg_cost,acc])
        if batch_id%100 == 0:
            print('pass :%d, Batch:%d, Cost:%d, Acc:%0.5f'%(pass_id, batch_id, train_cost[0],train_acc[0]))

    test_costs = []
    test_accs = []
    for batch_id,data in enumerate(test_reader()):
        test_cost,test_acc = exe.run(program=test_program,
                                     feed=feeder.feed(data),
                                     fetch_list=[avg_cost,acc])
        test_costs.append(test_cost[0])
        test_accs.append(test_acc[0])

    #计算平局损失和预测率
    test_cost = (sum(test_costs)/len(test_costs))
    test_acc = (sum(test_accs)/len(test_accs))
    print('Test:%d,Cost:%0.5f, ACC:%0.5f'%(pass_id, test_cost,test_acc))

if not os.path.exists(model_save_dir):
    os.makedirs(model_save_dir)
fluid.io.save_inference_model(model_save_dir,feeded_var_names=[words.name],
                              target_vars=[model],executor=exe)
print("训练模型保存完成")

TypeError: 'method' object cannot be interpreted as an integer